In [1]:
import math
import numpy.random as random
import numpy as np

# y = f(x) = sign(x) + noise -> y 等於 對x取符號（+- 1）加上雜訊 
# 雜訊為翻轉20%的資料

# 定義sign
def sign(value):
    return -1 if value <= 0 else 1

# 取得有雜音的y
def get_noised_y(x, noise_rate):
    y = np.ones(x.shape)
    for idx, value in enumerate(x):
        noise = (-1 if random.random() < noise_rate else 1)
        value = sign(value) * noise
        y[idx] = value
    return y

# 取得訓練的資料
def get_training_data(data_size=20):
    # 產生-1到1之間的隨機亂數
    X = random.uniform(-1, 1, size=data_size)
    
    # 產生y
    y = get_noised_y(X, 0.2)
    
    return (X, y)

# decision stumps
def get_hypotheis(s, x, theta):
    h = np.zeros(x.shape)
    for idx, value in enumerate(x):
        h[idx] = s * sign(value - theta)
    return h

In [2]:
def get_thetas(x):
    #在頭跟尾還有每個點的中間都放個theta
    x = np.sort(x)
    theta = np.zeros(x.shape[0] + 1)
    theta[0] = -1
    for idx in range(len(x) - 1):
        theta[idx+1] = (x[idx] + x[idx+1])/2.
    theta[len(x)] = 1
    return theta


In [3]:
exp_time = 5000
e_in = 0   
e_out = 0
for exp_idx in range(exp_time):
    X, y = get_training_data()
    thetas = get_thetas(X)
    e_in_plue_s = []
    e_in_minus_s = []
    for theta_idx, theta in enumerate(thetas):
        # s = 1
        h_plus_s = get_hypotheis(1, X, theta)
        h_plus_s_diff = h_plus_s - y
        # e_in = 將不同的row加起來取絕對值/(2*row的total數)
        e_in_plue_s.append((np.sum(np.absolute(h_plus_s_diff))/(len(h_plus_s_diff) * 2.)))
        
        # s = -1
        h_minus_s = get_hypotheis(-1, X, theta)
        h_minus_diff = h_minus_s - y
        e_in_minus_s.append((np.sum(np.absolute(h_minus_diff))/(len(h_minus_diff) * 2.)))
    min_value = min(min(e_in_plue_s), min(e_in_minus_s))
    theta = 0
    s = 1
    # 找出最小e_in的theta
    if min_value in e_in_plue_s:
        theta = thetas[e_in_plue_s.index(min_value)]
    else:
        s = -1
        theta = thetas[e_in_minus_s.index(min_value)]
    e_in += min_value
    e_out += (0.5 + 0.3 * s * (abs(theta) - 1))

print("e_in = " + "{:.2f}".format(e_in/exp_time)) 
print("e_out = " + "{:.2f}".format(e_out/exp_time))

# e_in = 0.17
# e_out = 0.26


e_in = 0.17
e_out = 0.26
